# LLM que faz resumo de páginas web

### Criando classe para website

In [3]:
from bs4 import BeautifulSoup
import requests

class Website:
    url: str
    title: str
    content: str

    def __init__(self, url):
        self.url = url
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "Sem título"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.content = soup.body.get_text(separator="\n", strip=True)

In [5]:
webcontent = Website(url="https://www.nba.com/")

In [6]:
webcontent.title

'The official site of the NBA for the latest NBA Scores, Stats & News. | NBA.com'

In [8]:
print(webcontent.content)

Navigation Toggle
Home
Tickets
NBA 2K26 Summer League Schedule
2025 Playoffs Schedule
Key Dates
Featured
NBA TV
Home
Top Stories
Summer League
Draft
Playoffs
Features
Events
Key Dates
Transactions
Future Starts Now
History
More
Stats Home
Dunk Score
Players
Teams
Leaders
Stats 101
Cume Stats
Lineups Tool
Media Central Game Stats
Draft
Quick Links
Contact Us
Atlantic
Boston Celtics
Brooklyn Nets
New York Knicks
Philadelphia 76ers
Toronto Raptors
Central
Chicago Bulls
Cleveland Cavaliers
Detroit Pistons
Indiana Pacers
Milwaukee Bucks
Southeast
Atlanta Hawks
Charlotte Hornets
Miami Heat
Orlando Magic
Washington Wizards
Northwest
Denver Nuggets
Minnesota Timberwolves
Oklahoma City Thunder
Portland Trail Blazers
Utah Jazz
Pacific
Golden State Warriors
LA Clippers
Los Angeles Lakers
Phoenix Suns
Sacramento Kings
Southwest
Dallas Mavericks
Houston Rockets
Memphis Grizzlies
New Orleans Pelicans
San Antonio Spurs
Players Home
2024-25 Audio Pronunciation Guide
Player Stats
Starting Lineups
Free 

### Instanciando LLM

In [9]:
import os
import openai
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

groq_api_key = os.getenv("GROQ_API_KEY")

client_groq = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_api_key
)

### Criando prompts

In [23]:
def get_messages(webcontent):
    system_prompt = """
    Você é o melhor em resumir conteúdos oriundos da internet.
    Você receberá o conteúdo de um website e você deverá resumir esse conteúdo de maneira objetiva e perfeita.
    Ignore conteúdos que parecem ser de navegação  pelo site. Devolva a resposta em markdown
    """

    user_prompt = """
    ** Conteúdo do website **

    {content_website}
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt.format(content_website=webcontent)}
    ]

    return messages

In [25]:
def get_summary(url):
    webcontent = Website(url=url)
    messages = get_messages(webcontent=webcontent.content)
    response = client_groq.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages
    )

    return response.choices[0].message.content


In [26]:
summary = get_summary(url="https://www.nba.com/")

In [27]:
from IPython.display import Markdown, display

display(Markdown(summary))

**Offseason Power Rankings: State of the NBA**

The 2025 NBA offseason is in full swing, and teams are making significant moves to bolster their rosters. In this article, we'll take a look at the top 15 teams in each conference and rank them based on their offseason performance.

**Western Conference**

1. Los Angeles Lakers - The Lakers have made significant improvements to their roster, adding Deandre Ayton and Marcus Smart to complement their core of LeBron James and Luka Dončić.
2. Golden State Warriors - The Warriors have rebuilt their roster around Steph Curry, adding D'Angelo Russell and Andrew Wiggins to the mix.
3. Denver Nuggets - The Nuggets have brought in Paul Millsap and Will Barton to bolster their frontcourt and improve their bench.
4. Utah Jazz - The Jazz have made significant changes to their roster, adding Bojan Bogdanovic and Mike Conley to support their young core.
5. Los Angeles Clippers - The Clippers have added Robert Covington and Marcus Morris to their roster, giving them a strong bench and more scoring punch.

**Eastern Conference**

1. Boston Celtics - The Celtics have re-signed Jayson Tatum and added Malcolm Brogdon to their roster, improving their bench and depth.
2. Brooklyn Nets - The Nets have brought in Blake Griffin and Spencer Dinwiddie to support their core of Kyrie Irving and Kevin Durant.
3. Philadelphia 76ers - The 76ers have added James Harden and Al Horford to their roster, giving them a strong starting lineup and improved bench.
4. Toronto Raptors - The Raptors have brought in Pascal Siakam and OG Anunoby to their roster, rounding out their young core.
5. Indiana Pacers - The Pacers have added Malcolm Brogdon and T.J. Warren to their roster, giving them more depth and scoring punch.

**Other notable moves**

* Spurs' De'Aaron Fox signs reported 4-year, $229 million extension.
* OKC shores up its core players with crucial extensions.
* Lakers sign Deandre Ayton and Marcus Smart to long-term extensions.
* Knicks sign Rui Hachimura to a reported 2-year deal.
* NBA Foundation celebrates its 5-year anniversary.

**Upcoming events**

* Offseason Power Rankings: State of the NBA will be updated weekly throughout the offseason.
* The 2025 NBA season is set to tip off on October 31.
* The NBA Draft is set to take place on June 22, 2026.